# Setup

In [2047]:
import pandas as pd
# from collections import defaultdict
# from operator import itemgetter
import numpy as np
# import re
# from fuzzywuzzy import process, fuzz

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.3f}'.format)

In [2048]:
dsd = "DSD2324_overview (2).xlsx"
preferences = "Teaching Preferences Questionnaire_S1_2023-24_raw.xlsx"
contract = "ta_contract_v2.xlsx"
bs_weights = "bs_courses_weights_FILLED.xlsx"
term = "S1"

# DEMAND - practical lessons

In [2049]:
dsd_df = pd.read_excel(dsd)

In [2050]:
dsd_df.head()

,ORDER,TERM,CYCLE,COURSE CODE,COURSE NAME,LANGUAGE,CLASS,SLOTS,WEIGHT,PROGRAM,FORMAT,INSTITUTIONAL ID,FACULTY ID,FACULTY NAME,FACULTY EMAIL,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,LAST UPDATED AT
0,1,S1,BSC,1104,Seminário de Economia Europeia,EN,T301A,35,1.000,NaN,NORMAL,16699,146,João Amador,jamador@novasbe.pt,NaN,NaN,NaN,NaN,NaN
1,2,S1,BSC,1104,Seminário de Economia Europeia,EN,T302A,35,1.000,NaN,NORMAL,16699,146,João Amador,jamador@novasbe.pt,NaN,NaN,NaN,NaN,NaN
2,3,S1,BSC,1104,Seminário de Economia Europeia,EN,T303A,35,1.000,NaN,NORMAL,16699,146,João Amador,jamador@novasbe.pt,NaN,NaN,NaN,NaN,NaN
3,4,S2,BSC,1104,Seminário de Economia Europeia,EN,T301A,40,1.000,NaN,NORMAL,16699,146,João Amador,jamador@novasbe.pt,NaN,NaN,NaN,NaN,NaN
4,5,S2,BSC,1104,Seminário de Economia Europeia,EN,T302A,40,1.000,NaN,NORMAL,16699,146,João Amador,jamador@novasbe.pt,NaN,NaN,NaN,NaN,NaN


In [2051]:
dsd_df["course"] = dsd_df["COURSE CODE"].astype(str) + " || " + dsd_df["COURSE NAME"].astype(str) + " || " + dsd_df["TERM"].astype(str) + " || " + dsd_df["LANGUAGE"].astype(str)

# ADDED!
faculty_list = dsd_df[dsd_df["COURSE NAME"] != "Stata"]["FACULTY EMAIL"].unique()

if term == "S1":
    selected_terms = ["S1", "T1", "T2"]
    dsd_df = dsd_df[dsd_df['TERM'].isin(selected_terms)]
else:
    selected_terms = ["S2", "T3", "T4"]
    dsd_df = dsd_df[dsd_df['TERM'].isin(selected_terms)]

# ADDED!
teorico_practicas = dsd_df[(dsd_df["FACULTY NAME"].isna()) & (dsd_df["CYCLE"] == "BSC")]["COURSE NAME"].unique()
dsd_df = dsd_df.drop(dsd_df[(dsd_df["COURSE NAME"].isin(teorico_practicas)) & (dsd_df["FACULTY NAME"].notna())].index)

In [2052]:
faculty_list

array(['jamador@novasbe.pt', 'afreitas@novasbe.pt',
       'victoire.girard@novasbe.pt', 'vascos@novasbe.pt',
       'eduardo.costa@novasbe.pt', 'ppc@novasbe.pt', nan,
       'priscila.oliveira@berkeley.edu', 'claudia.alves@novasbe.pt',
       'ppbarros@novasbe.pt', 'jmcosta@novasbe.pt | sharmin@novasbe.pt',
       'joao.silva@novasbe.pt', 'peralta@novasbe.pt',
       'pedro.martins@novasbe.pt',
       'pedro.brinca@novasbe.pt | joao.duarte@novasbe.pt',
       'lisbeth.rodrigues@novasbe.pt', 'milton.sousa@novasbe.pt',
       'youtha.cuypers@novasbe.pt', 'paulo.faroleiro@novasbe.pt',
       'miguel.munoz.duarte@novasbe.pt', 'mariia.murasheva@novasbe.pt',
       'lpereira@novasbe.pt', 'filipa.fonseca@novasbe.pt',
       'workprojects@novasbe.pt', 'bernardo.pimentel@novasbe.pt',
       'joao.m.filipe@novasbe.pt', 'carlos.cardoso@novasbe.pt',
       'ana.marques@uea.ac.uk', 'raugusto@novasbe.pt',
       'marta.almeida@novasbe.pt', 'melissa.prado@novasbe.pt',
       'niklasflamang@berkeley.

In [2053]:
agg_functions = {
    'CLASS': 'count',
    'SLOTS': np.sum
    # 'COURSE NAME': 'first'  # Add 'COURSE NAME' to the aggregation functions
}

# ADDED! "CYCLE"
output_1 = dsd_df.groupby(['COURSE NAME', 'TERM', 'COURSE CODE', 'LANGUAGE', 'CYCLE']).agg(agg_functions).reset_index()
output_1 = output_1.rename(columns={'CLASS': 'Nº CLASSES', 'SLOTS': 'Nº STUDENTS'})


output_1.head()

,COURSE NAME,TERM,COURSE CODE,LANGUAGE,CYCLE,Nº CLASSES,Nº STUDENTS
0,Advanced Data Analysis,S1,2597,EN,MST,2,140
1,Advanced Financial Management,S1,2581,EN,MST,4,360
2,Advanced Marketing,S1,2430,EN,MST,8,480
3,Advanced Maths and Stats for Economics,T1,4508,EN,MST,1,999
4,Advanced Methods,S1,270107,EN,PHD,1,15


In [2054]:
dsd_df.TERM.value_counts(dropna=False)

TERM
S1    263
T2    127
T1    123
Name: count, dtype: int64

In [2055]:
course_demand = dsd_df.groupby(['course']).agg(agg_functions).reset_index()
course_demand = course_demand.rename(columns={'CLASS': 'number_classes', 'SLOTS': 'number_students'})

course_demand.head()

,course,number_classes,number_students
0,107789 || Innovation for Impact || T2 || EN,2,140
1,1104 || Seminário de Economia Europeia || S1 |...,3,105
2,1108 || Economia do Desenvolvimento || S1 || EN,2,70
3,1112 || Economia Industrial || S1 || EN,3,126
4,1113 || Economia Global I || S1 || EN,2,84


# SUPPLY - TA hours (current contract)

In [2056]:
contract = pd.read_excel(contract)

contract = contract[["TA", "CONTRACT"]]
contract["TA"] = contract["TA"].str.lower()

# Zero contracts
# ADDED! Caps "CONTRACT", remove faculty
zero_contracts = contract[contract['CONTRACT'] == 0]["TA"].unique()
contract = contract[contract['CONTRACT'] != 0]
contract = contract[~contract['TA'].isin(faculty_list)]


In [2057]:
zero_contracts

array(['silvia.guerra@novasbe.pt', 'joana.matos@novasbe.pt'], dtype=object)

In [2058]:
# Check for duplicates
contract[contract.duplicated()].TA.count()

0

In [2059]:
contract_emails = contract.TA.unique()

In [2060]:
contract.head()

,TA,CONTRACT
0,abrao.kulaif@novasbe.pt,0.500
1,afonso.ribeiro@novasbe.pt,0.500
2,afonso.serrano@novasbe.pt,0.250
3,afonso.castro@novasbe.pt,0.500
4,afonso.duarte@novasbe.pt,0.175


# SUPPLY - TA preferences

## A. Preferences

In [2061]:
# Read the Excel file with the second row as the header
preferences_df = pd.read_excel(preferences, header=1)

# Sort the DataFrame by "End Date" column in descending order
preferences_df = preferences_df.sort_values(by='End Date', ascending=False)

# Rename the column to "TA"
preferences_df.rename(columns={'Please write your E-mail @novasbe.pt': 'TA'}, inplace=True)

# Convert the values in the "TA" column to lowercase
preferences_df['TA'] = preferences_df['TA'].str.lower()

# Create a new dataframe with column names and zero-indexed column numbers
column_df = pd.DataFrame({'Column Name': preferences_df.columns,
                          'Column Number': range(len(preferences_df.columns))})

# Remove TAs with zero_contracts
preferences_df = preferences_df[~preferences_df["TA"].isin(zero_contracts)]

# ADDED! Remove TAs wicha are faculty
preferences_df = preferences_df[~preferences_df["TA"].isin(faculty_list)]

# Create a mask to identify duplicates in the "TA" column
duplicates_mask = preferences_df.duplicated(subset='TA', keep=False)
preferences_duplicates = preferences_df[duplicates_mask]
preferences_duplicates = preferences_duplicates.sort_values(by='End Date', ascending=False)

preferences_duplicates_last = preferences_duplicates.drop_duplicates(subset='TA', keep='first').copy()

# Create a mask to check if columns 31:81 or 83:347 have values
value_mask = preferences_duplicates.iloc[:, 31:81].notnull().any(axis=1) | preferences_duplicates.iloc[:, 83:347].notnull().any(axis=1)
preferences_duplicates_values = preferences_duplicates[value_mask]
preferences_duplicates_values = preferences_duplicates_values.drop_duplicates(subset='TA', keep='first').copy()

# Drop duplicates based on the "TA" column
preferences_df = preferences_df[~duplicates_mask]

# Drop duplicates based on the "Full Name" column while keeping the row with the most recent "End Date" (ex. Franziska wrong)
preferences_df = preferences_df.drop_duplicates(subset='Full Name', keep='first')

# Create a new DataFrame with columns from preferences_duplicates_last
preferences_df_final = preferences_duplicates_last.copy()

# Get the relevant columns from preferences_duplicates_values
preference_columns = preferences_duplicates_values.columns[31:81].tolist() + preferences_duplicates_values.columns[83:347].tolist()

# Update the values in preferences_df_final using values from preferences_duplicates_values for preference_columns
preferences_df_final.set_index('TA', inplace=True, drop=False)
preferences_duplicates_values.set_index('TA', inplace=True, drop=False)
preferences_df_final.loc[preferences_duplicates_values.index, preference_columns] = preferences_duplicates_values[preference_columns].values

# Concatenate the remaining columns from preferences_df to preferences_df_final
preferences_df_final = pd.concat([preferences_df_final, preferences_df])

# Sort the final DataFrame by "End Date" column in descending order
preferences_df_final.sort_values(by='End Date', ascending=False, inplace=True)

# Reset the index of the final DataFrame
preferences_df_final.reset_index(drop=True, inplace=True)

# Drop duplicates based on the "Full Name" column while keeping the row with the most recent "End Date" (ex. Franziska wrong )
preferences_df_final.drop_duplicates(subset='Full Name', keep='first', inplace=True)

# Create a mapping of original column names to new column names (course ID as integer)
mapping = {}
# course_full_codes = []

for column_name in preference_columns:
    # Extract the course ID number from the column name
    # course_id = column_name.split(' || ')[0].split(' - ')[3] + " || " + column_name.split(' || ')[1] + ' || ' +  column_name.split(' || ')[2].split(' - ')[0]
    course_id = column_name.split(' || ')[0].split(' - ')[3] + " || "  + column_name.split(' || ')[0].split(' - ')[4]+ " || " + column_name.split(' || ')[1] + ' || ' +  column_name.split(' || ')[2].split(' - ')[0]
    
    # Extract full course ID for matching later
    # course_full_codes.append(column_name.split(' || ')[0].split(' - ')[3] + " || "  + column_name.split(' || ')[0].split(' - ')[4]+ " || " + column_name.split(' || ')[1] + ' || ' +  column_name.split(' || ')[2].split(' - ')[0])
    
    # Map the original column name to the course ID
    mapping[column_name] = course_id

# Remove duplicates
# course_full_codes = list(set(course_full_codes))

# Rename the columns using the mapping
preferences_df_final.rename(columns=mapping, inplace=True)

# Drop columns with list of courses (redundant) [30, 81, and 82]
preferences_df_final.drop(columns=preferences_df_final.iloc[:,[30, 81, 82]], inplace=True)

# OUTPUT #2
# Filter the DataFrame for rows where "Do you intend to continue your collaboration with Nova SBE next semester as Teaching Assistant?" 
# (column 20) is not equal to "No"
ta_exits = preferences_df_final[preferences_df_final.iloc[:, 19] == "No"].iloc[:, [17, 18, 20]]
ta_exits = ta_exits.rename(columns={ta_exits.columns[-1]: "Comments"}).sort_values("Full Name")

# ta_exits.to_excel("ta_exits_v1.xlsx", index=False)
ta_exits_list = ta_exits.TA.unique()

preferences_df_final = preferences_df_final[preferences_df_final.iloc[:, 19] != "No"]

In [2062]:
column_17 = column_df[column_df['Column Name'] == "Full Name"].iloc[0]["Column Number"]
column_18 = column_df[column_df['Column Name'] == "TA"].iloc[0]["Column Number"]

continue_str = "Do you intend to continue your collaboration with Nova SBE next semester"
continue_just_str = "Please write here a short justification on why you do not intend to continue"
bs_or_ms_str = "Do you prefer to be assigned to Bachelor’s or Master's courses?"
load_availability_str = "What is your availability in terms of workload and contract percentage for the next semester?"
ms_student_str = "In the upcoming semester, are you going to be a Nova SBE student?"
phd_restrictions_str = "Being a PhD student, do you have any constraint in the number of teaching hours or contract percentage"
new_workload_str = "What is your availability in terms of workload and contract percentage for the next semester"

column_19 = column_df[column_df['Column Name'].str.startswith(continue_str)].iloc[0]["Column Number"]
column_20 = column_df[column_df['Column Name'].str.startswith(continue_just_str)].iloc[0]["Column Number"]
column_21 = column_df[column_df['Column Name'].str.startswith(ms_student_str)].iloc[0]["Column Number"]
column_22 = column_df[column_df['Column Name'].str.startswith(bs_or_ms_str)].iloc[0]["Column Number"]
column_23 = column_df[column_df['Column Name'].str.startswith(phd_restrictions_str)].iloc[0]["Column Number"]
column_27 = column_df[column_df['Column Name'].str.startswith(load_availability_str)].iloc[0]["Column Number"]
column_28 = column_27 + 1
column_29 = column_28 + 1 # Be careful! This assumes there are TWO text boxes for available workload percentage

bs_str = "Please choose below your teaching preferences for Bachelor Courses."
column_30 = column_df[column_df['Column Name'].str.startswith(bs_str)].iloc[0]["Column Number"]
column_31 = column_30 + 1 # BE careful! This assumes there is ONE open text columns for bachelors preferences

ms_str = "Please choose below your teaching preferences for Masters Courses (grading)."
column_81 = column_df[column_df['Column Name'].str.startswith(ms_str)].iloc[0]["Column Number"]
column_82 = column_81 + 1 # BE careful! This assumes there are TWO open text columns for master preferences
column_83 = column_81 + 2 # BE careful! This assumes there are TWO open text columns for master preferences

print(column_17, column_18, column_19, column_20, column_21, column_22, column_23, column_27, column_28, column_29, column_30, column_31, column_81, column_82, column_83)

17 18 19 20 21 22 23 27 28 29 30 31 81 82 83


In [2063]:
column_df.iloc[column_28]

Column Name      What is your availability in terms of workload...
Column Number                                                   28
Name: 28, dtype: object

In [2064]:
print(f"Number of TAs who are leaving: {len(ta_exits_list)}")

Number of TAs who are leaving: 17


In [2065]:
# Check that can use the email as key/ ID
preferences_df_final["TA"].isnull().sum()

0

In [2066]:
# Check there are no duplicates names
preferences_df_final[preferences_df_final["Full Name"].duplicated()]["Response ID"].count()

0

In [2067]:
# Check there are no duplicates answers
preferences_df_final[preferences_df_final["TA"].duplicated()]["Response ID"].count()

0

In [2068]:
responded_number = preferences_df_final.shape[0] 
print(f"Number of TAs willing to continue who responded: {responded_number}")
display(preferences_df_final[["TA", "Full Name"]].head())

Number of TAs willing to continue who responded: 166


,TA,Full Name
0,beatriz.f.ferreira@novasbe.pt,Beatriz Frazão Ferreira
1,sara.b.lopes@novasbe.pt,Sara de Louxembourg Henriques Borges Lopes
3,marcio.silva@novasbe.pt,Marcio Nunes da Silva
4,ana.lourenco@novasbe.pt,Ana Filipa Duarte Lourenço
5,fabian.wassmann@novasbe.pt,Fabian Wassmann


In [2069]:
# OUTPUT #N
preferences_df_final[~preferences_df_final.iloc[:,-1].isna()].iloc[:, [17, 18, -1]]

,Full Name,TA,Please write below your observations:
0,Beatriz Frazão Ferreira,beatriz.f.ferreira@novasbe.pt,Only for the course of Business Strategy and P...
3,Marcio Nunes da Silva,marcio.silva@novasbe.pt,"I can only handle one course per semester, sin..."
8,Andre Daniel Vinhas Nunes,andre.nunes@novasbe.pt,"""Economics of education"" and ""Competition Poli..."
9,Franziska Hittmair,hittmair.franziska@novasbe.pt,Global Strategic Management for CEMS
18,Valter Miguel Sobral Nóbrega,valter.nobrega@novasbe.pt,Wiseflow preference
21,Afonso Maria Jardim Gonçalves Teixeira Duarte,afonso.duarte@novasbe.pt,I would like to be assistant also in Risk Mana...
24,Gonçalo Maria de Abreu Peixoto Cordeiro de Sousa,goncalo.sousa@novasbe.pt,I would like to continue to work with Prof Ema...
30,Rafael de Almeida Sequeira,rafael.sequeira@novasbe.pt,If I am allocated to Investments (full course)...
33,Maria José Rodrigues Aniceto,maria.aniceto@novasbe.pt,I would like to keep grading financial managem...
34,Afonso Saraiva Câmara Leme,afonso.leme@novasbe.pt,First option is Economics of Education (shows ...


In [2070]:
# OUTPUT #N
preferences_df_final[~preferences_df_final["TA"].isin(contract_emails)][["TA", "Full Name"]]

,TA,Full Name
61,dd@novasbe.pt,dddd
81,pedrobrinca@novasbe.pt,Pedro Brinca
111,xxx@novasbe.pt,xxx
114,aa@novasbe.pt,aaa
134,test@novasbe.pt,test


In [2071]:
# Get the course columns
course_columns = preferences_df_final.columns[30:-1]

# Create a new DataFrame for the adapted format
adapted_df = pd.DataFrame(columns=["TA", "course", "preference", "preference_type"])

# Define the translation mapping for column 22 values
translation_mapping = {
    "Masters' Courses": 2,
    "Bachelors' Courses": 0,
    "Indifferent": 1,
    pd.NaT: 1  # Assuming NaN values should also be considered "Indifferent"
}

# Iterate over the course columns
for course in course_columns:
    # Check if the course has already been processed
    if course in adapted_df["course"].unique():
        continue

    # Get the duplicate columns for the current course
    duplicate_columns = [col for col in course_columns if col != course and col.endswith(course)]

    # Combine the duplicate columns into a single column
    combined_column = preferences_df_final[[course] + duplicate_columns].ffill(axis=1).iloc[:, -1]

    # Filter the DataFrame for non-null values in the combined column
    non_null_mask = combined_column.notnull()
    non_null_df = preferences_df_final[non_null_mask]

    # Get the teacher names and their corresponding preference rankings for the current course
    teacher_names = non_null_df["TA"]
    preference_rankings = combined_column[non_null_mask]

    # Get the corresponding preference types based on the translation mapping
    preference_types = non_null_df.iloc[:, 22].map(translation_mapping)

    # Create a DataFrame for the current course, preference rankings, and preference types
    course_df = pd.DataFrame({"TA": teacher_names, "course": [course] * len(teacher_names),
                              "preference": preference_rankings, "preference_type": preference_types})

    # Concatenate course_df with adapted_df
    adapted_df = pd.concat([adapted_df, course_df], ignore_index=True)
    
    # Create the 'masters_course' column based on the condition
    adapted_df['masters_course'] = adapted_df['course'].apply(lambda x: 0 if x.split(' ')[0].startswith('1') else 1)

    # Convert "preference" column to integers
    adapted_df['preference'] = adapted_df['preference'].astype(np.int8)

    # Remove preferences above 5
    adapted_df = adapted_df[adapted_df['preference']<=5]

In [2072]:
# Check there are no preferences above 5 and below 1
print(adapted_df['preference'].min())
print(adapted_df['preference'].max())

1
5


In [2073]:
completed_preferences = adapted_df["TA"].unique()
completed_preferences_number = len(completed_preferences)
print(f"Number of TAs with completed preferences: {completed_preferences_number}")
# display(completed_preferences)

Number of TAs with completed preferences: 154


In [2074]:
# OUTPUT #6
# ADDED! removed :-1
# adapted_df.iloc[:,:-1].head()
adapted_df.head()

,TA,course,preference,preference_type,masters_course
0,marlon.e.francisco@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,5,1,0
1,luis.pereira@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,5,0,0
2,stefano.grancini@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,3,1,0
3,joao.quelhas@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,5,0,0
4,luis.faria@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,1,2,0


In [2075]:
adapted_df[adapted_df["TA"]=="bernardo.costa@novasbe.pt"]

,TA,course,preference,preference_type,masters_course
167,bernardo.costa@novasbe.pt,1312 || Data Analysis and Probability || S1 || EN,3,1,0
179,bernardo.costa@novasbe.pt,1313 || Statistics for Economics and Managemen...,2,1,0
187,bernardo.costa@novasbe.pt,1314 || Econometrics || S1 || EN,4,1,0
193,bernardo.costa@novasbe.pt,1318 || Introduction to Programming || S1 || EN,1,1,0
499,bernardo.costa@novasbe.pt,2477 || Introduction to Programming || S1 || EN,2,1,1
558,bernardo.costa@novasbe.pt,2597 || Advanced Data Analysis || S1 || EN,1,1,1
566,bernardo.costa@novasbe.pt,2609 || Data Visualization for Business Analyt...,4,1,1
610,bernardo.costa@novasbe.pt,2659 || Data Curation for Business Analytics |...,3,1,1


## B. New contract (preferences)

In [2076]:
contact_list = contract[(~contract.TA.isin(completed_preferences)) & (~contract.TA.isin(ta_exits_list))]
print(contact_list.shape[0])
display(contact_list)
contact_list.to_excel("ta_contact_list_v1.xlsx", index=False)

29


,TA,CONTRACT
1,afonso.ribeiro@novasbe.pt,0.500
3,afonso.castro@novasbe.pt,0.500
20,ana.silva@novasbe.pt,0.200
24,andre.tome@novasbe.pt,0.100
32,beatriz.marques@novasbe.pt,0.250
33,beatriz.bento@novasbe.pt,0.125
47,claudia.castico@novasbe.pt,0.375
54,daniele.gamberoni@novasbe.pt,0.200
56,diogo.v.rebelo@novasbe.pt,0.400
70,hittmair.franziska@novasbe.pt,0.250


In [2077]:
def round_to_closest(value):
    if pd.isnull(value):
        return np.nan
    else:
        # rounded_value = round(value * 8) / 8  # Round to the nearest multiple of 0.125
        capped_value = min(value, 0.5)  # Cap the value at 0.5
        capped_value = max(value, 0.1)  # Cap the value at 0.1
        return capped_value

# Clean the "load_requested" column
def clean_percentage(value):
    if pd.isnull(value):
        return value
    elif isinstance(value, str):
        # Check if the value contains only text characters
        if value.isalpha():
            return np.nan

        # Extract numeric values from string
        numeric_value = ''.join(filter(str.isdigit, value))

        if numeric_value == '':
            return np.nan

        if numeric_value == '100':
            return 100

        if len(numeric_value) >= 2:
            integer_part = numeric_value[:2]
            decimal_part = numeric_value[2:]
            return float(integer_part + '.' + decimal_part)

        return np.nan

    elif isinstance(value, (int, float)):
        return float(value) / 100

    return value

def decrease_contract_level(value):
    return value - 0.125


mapping = {
    "I want to increase the contract percentage/workload in the next semester (please specify the desired contract percentage level)": 1,
    "I want to keep the same contract percentage/workload as this semester": 0,
    "I want to reduce the contract percentage/workload in the next semester (please specify the desired contract percentage level)": -1,
    pd.NaT: 0
}

mapping_21 = {
    "Yes, I am a PhD student": 0,
    "Yes, I will be a Masters student but not doing any courses, only the Work Project": 0,
    "Yes, I will be a Masters student and I will be doing at least one more course": 1,
    "No": 0,
    pd.NaT: 0
}

mapping_23 = {
    "Yes, I have some other constraints that limit my teaching hours/workload (please specify the reason and the limit)": 1,
    "Yes, I have a FCT scholarship that limits my weekly teaching hours to 4h per week": 1,
    "No": 0,
    pd.NaT: 0
}

################################################################################################################################################

mask = preferences_df_final.iloc[:, 27].notna()
new_contract = preferences_df_final[mask].iloc[:, [18, 21, 23, 27, 28, 29]]

new_contract.columns = ['TA', 'master_student', 'PhD_restrictions', 'change_load', 'new_contract_decreased_load', 'new_contract_increased_load']
new_contract['change_load'] = new_contract['change_load'].map(mapping)
new_contract['master_student'] = new_contract['master_student'].map(mapping_21).fillna(0).astype(int)
new_contract['PhD_restrictions'] = new_contract['PhD_restrictions'].map(mapping_23).fillna(0).astype(int)

# Convert TA column to lowercase
new_contract['TA'] = new_contract['TA'].str.lower()

new_contract['new_contract_decreased_load'] = new_contract['new_contract_decreased_load'].apply(clean_percentage) / 100
new_contract['new_contract_increased_load'] = new_contract['new_contract_increased_load'].apply(clean_percentage) / 100

# Merge "new_contract_decreased_load" and "new_contract_increased_load" into "load_requested"
new_contract['load_requested'] = new_contract[['new_contract_decreased_load', 'new_contract_increased_load']].mean(axis=1)
new_contract['load_requested'] = new_contract['load_requested'].apply(round_to_closest)

# Drop "new_contract_decreased_load" and "new_contract_increased_load" columns
new_contract.drop(columns=['new_contract_decreased_load', 'new_contract_increased_load'], inplace=True)

In [2078]:
# OUTPUT #3
new_contract[new_contract.change_load !=0].sort_values(by="change_load",ascending=False)

,TA,master_student,PhD_restrictions,change_load,load_requested
3,marcio.silva@novasbe.pt,0,1,1,NaN
98,rodrigo.duarte@novasbe.pt,0,0,1,0.500
180,yulia.vachevskikh@novasbe.pt,0,1,1,1.000
173,diogo.lima@novasbe.pt,0,0,1,0.250
170,fernanda.rates@novasbe.pt,0,0,1,0.500
167,daniel.silva@novasbe.pt,0,0,1,0.500
164,isabel.silva.carvalho@novasbe.pt,0,0,1,NaN
154,anastasiia.smirnova@novasbe.pt,0,0,1,0.500
140,hugo.sousa@novasbe.pt,0,0,1,0.500
138,pamela.pacciani@novasbe.pt,0,0,1,0.500


In [2079]:
# Check new_contract_load is formatted correctly
# new_contract.load_requested.value_counts()

In [2080]:
number_TAS = len(new_contract[new_contract.change_load.isin([1,-1])])
print(f"Number of TAs who want to change their contract: {number_TAS}")

Number of TAs who want to change their contract: 43


In [2081]:
all_contracts = contract.merge(new_contract, how="left", on="TA")

# Filter rows where change_load is not equal to 0
filtered_contracts = all_contracts[all_contracts['change_load'] != 0].copy()

# Decrease contract to load_requested for rows where change_load is -1
filtered_contracts.loc[filtered_contracts['change_load'] == -1, 'new_contract'] = filtered_contracts['load_requested']
# ADDED! Caps "CONTRACT"
filtered_contracts.loc[(filtered_contracts['change_load'] == -1) & (filtered_contracts['load_requested'].isnull()), 'new_contract'] = filtered_contracts.apply(lambda row: decrease_contract_level(row['CONTRACT']), axis=1)

# # Increase contract for no restrictions
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 0), 'new_contract'] = filtered_contracts['load_requested']
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 0) & (filtered_contracts['load_requested'].isnull()), 'new_contract'] = np.minimum(filtered_contracts['contract'] * 2, 0.5)

# Apply conditions for updating new_contract based on change_load and restrictions
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 1), 'new_contract'] = filtered_contracts['contract']
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 2), 'new_contract'] = np.maximum(filtered_contracts['contract'], 0.25)

# Fill NaN values with the original contract value
# ADDED! Caps "CONTRACT"
filtered_contracts['new_contract'].fillna(filtered_contracts['CONTRACT'], inplace=True)

# Create a new column "new_contract" in the original DataFrame with NaN values
all_contracts['new_contract'] = np.nan

# Update the "new_contract" column in the original DataFrame with the filtered values
all_contracts.update(filtered_contracts[['new_contract']])
# ADDED! Caps "CONTRACT"
all_contracts['new_contract'].fillna(all_contracts['CONTRACT'], inplace=True)

# Round the "new_contract" values to the closest contract level (0.125, 0.25, 0.375, 0.5)
# all_contracts['new_contract'] = all_contracts['new_contract'].apply(round_to_closest)

# Drop emails which currently do not have a contract (ex. pedro.brinca)
# ADDED! Caps "CONTRACT"
all_contracts = all_contracts[all_contracts.CONTRACT.notna()]

In [2082]:
# Check there are no TAs wihout current contract
all_contracts[all_contracts.CONTRACT.isnull()].TA.to_list()

[]

In [2083]:
all_contracts = all_contracts[["TA", "new_contract", "master_student"]]

## C. Merge SUPPLY and DEMAND

In [2084]:
ta_preferences = adapted_df.merge(all_contracts, how="left", on="TA", indicator=True)
ta_preferences.head()

,TA,course,preference,preference_type,masters_course,new_contract,master_student,_merge
0,marlon.e.francisco@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,5,1,0,0.500,0.000,both
1,luis.pereira@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,5,0,0,0.500,1.000,both
2,stefano.grancini@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,3,1,0,0.250,0.000,both
3,joao.quelhas@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,5,0,0,0.375,0.000,both
4,luis.faria@novasbe.pt,1104 || Seminar in European Economics || S1 || EN,1,2,0,0.150,0.000,both


In [2085]:
# Check for non-matching TAs
non_matching_values = ta_preferences[ta_preferences['_merge'] != 'both']
ta_preferences.drop(columns=["_merge"], inplace=True)
display(non_matching_values)

,TA,course,preference,preference_type,masters_course,new_contract,master_student,_merge


In [2086]:
market = ta_preferences.merge(course_demand, how="left", on="course", indicator=True)

# market['number_classes'] = pd.to_numeric(market['number_classes'], errors='coerce').astype(pd.Int64Dtype())
# market['number_students'] = pd.to_numeric(market['number_students'], errors='coerce').astype(pd.Int64Dtype())

In [2087]:
non_matching_values = market[market['_merge'] != 'both']
market.drop(columns=["_merge"], inplace=True)

non_matching_courses = non_matching_values[["course"]].drop_duplicates()
non_matching_courses = non_matching_courses.copy()
non_matching_courses[["course_code", "course_name", "period", "language"]] = non_matching_courses["course"].str.split(" \|\| ", expand=True)

course_demand_extended = course_demand.copy()
course_demand_extended[["course_code", "course_name", "period", "language"]] = course_demand["course"].str.split(" \|\| ", expand=True)
course_demand_extended = course_demand_extended[["course", "course_code", "course_name", "period", "language"]]

In [2088]:
# Initialize an empty DataFrame to store the concatenated results
concatenated_matches = pd.DataFrame()

# Merge on 'course_code', 'period', and 'language'
merged_courses = pd.merge(non_matching_courses, course_demand_extended, on=["course_code", "period", "language"], how="left", suffixes=("", "_new"))
still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# Merge on 'course_name', 'period', and 'language'
merged_courses = pd.merge(still_unmatched, course_demand_extended, on=["course_name", "period", "language"], how="left", suffixes=("", "_new"))
still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# Merge on 'course_code' and 'period'
merged_courses = pd.merge(still_unmatched, course_demand_extended, on=["course_code", "period"], how="left", suffixes=("", "_new"))
still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# # [DANGER! Might include courses from different semester] Merge on 'course_code' and 'language' 
# merged_courses = pd.merge(still_unmatched, course_demand_extended, on=["course_code", "language"], how="left", suffixes=("", "_new"))
# still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
# concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# Merge concatenated_matches on the market DataFrame to add the "course_new" column
market = pd.merge(market, concatenated_matches[["course", "course_new"]], on=["course"], how="left")
market["course_new"].fillna(market["course"], inplace=True)
market.rename(columns={"course": "course_old"}, inplace=True)
market.drop(columns=["course_old"], inplace=True)
market.rename(columns={"course_new": "course"}, inplace=True)

# Merge market and course_demand on "course" column
merged_market = pd.merge(market, course_demand[["course", "number_classes", "number_students"]], on="course", how="left", suffixes=("", "_demand"))

# Fill NaN values in number_classes and number_students columns
merged_market["number_classes"].fillna(merged_market["number_classes_demand"], inplace=True)
merged_market["number_students"].fillna(merged_market["number_students_demand"], inplace=True)

# Drop the unnecessary columns
merged_market.drop(columns=["number_classes_demand", "number_students_demand"], inplace=True)

In [2089]:
# OUTPUT #4
# Display the updated merged_market DataFrame
no_matches_final = merged_market[(merged_market.number_classes.isna()) | (merged_market.number_students.isna())][["course"]]
no_matches_final = no_matches_final.drop_duplicates()
display(no_matches_final)

# Drop these courses
merged_market.dropna(subset=["number_classes", "number_students"], inplace=True)

,course
228,2134 || Economics of Education || T1 || EN
332,2260 || Nova Students Portfolio || A || EN
360,2329 || Consumer and Managerial Decision Makin...
365,2330 || Consumer Behavior || T2 || EN
470,2452 || Business Strategy and Practice || T2 |...
560,2604 || Brand Management for Hospitality || T2...
569,2621 || Algorithmic Governance || T1 || EN
582,2642 || Principles of Shopper Marketing || T2 ...
586,2645 || Tourism Marketing || S1 || EN
604,2653 || Impact Makers || S1 || EN


## D. Compute capacity

In [2090]:
# Create the "semester" column based on the condition
merged_market['semester'] = merged_market['course'].apply(lambda x: 1 if x.split(' || ')[2].startswith('S') else 0)
# merged_market['ms_capacity'] = merged_market['new_contract'] * 36

display(merged_market[(merged_market['semester'].isnull() | merged_market['masters_course'].isnull())])

# Define a function to apply the conditions
def calculate_weight(row):
    if pd.isnull(row['semester']) or pd.isnull(row['masters_course']):
        return np.nan
    elif row['semester'] == 1 and row['masters_course'] == 1:
        return ((row['number_students'] * 2.33) / 16) / 36
    elif row['semester'] == 0 and row['masters_course'] == 1:
        return ((row['number_students'] * 1.25) / 16) / 36
    else:
        return np.nan

# Apply the function to create the 'ms_weight' column
merged_market['weight'] = merged_market.apply(calculate_weight, axis=1)

# Set 'ms_capacity' to NaN when 'masters_course' is 0
# merged_market.loc[merged_market['masters_course'] == 0, 'ms_capacity'] = np.nan
# merged_market.loc[merged_market['masters_course'] == 0, 'ms_weight'] = np.nan

,TA,preference,preference_type,masters_course,new_contract,master_student,number_classes,number_students,course,semester


In [2091]:
merged_market[merged_market.TA=="bernardo.costa@novasbe.pt"]

,TA,preference,preference_type,masters_course,new_contract,master_student,number_classes,number_students,course,semester,weight
167,bernardo.costa@novasbe.pt,3,1,0,0.333,0.000,2.000,90.000,1312 || Análise de Dados e Probabilidade || S1...,1,NaN
179,bernardo.costa@novasbe.pt,2,1,0,0.333,0.000,8.000,340.000,1313 || Estatística para Economia e Gestão || ...,1,NaN
187,bernardo.costa@novasbe.pt,4,1,0,0.333,0.000,2.000,76.000,1314 || Econometria || S1 || EN,1,NaN
193,bernardo.costa@novasbe.pt,1,1,0,0.333,0.000,2.000,90.000,1318 || Programação || S1 || EN,1,NaN
499,bernardo.costa@novasbe.pt,2,1,1,0.333,0.000,4.000,320.000,2477 || Introduction to Programming || S1 || EN,1,1.294
558,bernardo.costa@novasbe.pt,1,1,1,0.333,0.000,2.000,140.000,2597 || Advanced Data Analysis || S1 || EN,1,0.566
566,bernardo.costa@novasbe.pt,4,1,1,0.333,0.000,2.000,140.000,2609 || Data Visualization for Business Analyt...,0,0.304
610,bernardo.costa@novasbe.pt,3,1,1,0.333,0.000,2.000,130.000,2659 || Data Curation for Business Analytics |...,0,0.282


In [2092]:
merged_market.masters_course.value_counts(dropna=False)

masters_course
1    394
0    228
Name: count, dtype: int64

### Get file course list to manually input the weights

In [2093]:
course_demand_extended['masters_course'] = course_demand_extended['course'].apply(lambda x: 0 if x.split(' ')[0].startswith('1') else 1)
course_demand_extended_bs = course_demand_extended[course_demand_extended.masters_course==0]
course_demand_extended_bs = course_demand_extended_bs.drop(columns=["masters_course"])
course_demand_extended_bs["weight"] = ""
course_demand_extended_bs.to_excel("course_demand_extended_bs.xlsx", index=False)

### Read the manually inputed weights

In [2094]:
bs_weights_df = pd.read_excel(bs_weights)[["course", "weight"]]
bs_weights_df["weight"] = bs_weights_df["weight"] * 0.125
bs_weights_df.weight.value_counts(dropna=False)

weight
0.125    89
0.333     9
0.167     3
0.250     1
Name: count, dtype: int64

In [2095]:
final_market = pd.merge(merged_market, bs_weights_df, on=["course"], how="left", suffixes=("", "_bs"), indicator=True)
final_market.head()

,TA,preference,preference_type,masters_course,new_contract,master_student,number_classes,number_students,course,semester,weight,weight_bs,_merge
0,marlon.e.francisco@novasbe.pt,5,1,0,0.500,0.000,3.000,105.000,1104 || Seminário de Economia Europeia || S1 |...,1,NaN,0.125,both
1,luis.pereira@novasbe.pt,5,0,0,0.500,1.000,3.000,105.000,1104 || Seminário de Economia Europeia || S1 |...,1,NaN,0.125,both
2,stefano.grancini@novasbe.pt,3,1,0,0.250,0.000,3.000,105.000,1104 || Seminário de Economia Europeia || S1 |...,1,NaN,0.125,both
3,joao.quelhas@novasbe.pt,5,0,0,0.375,0.000,3.000,105.000,1104 || Seminário de Economia Europeia || S1 |...,1,NaN,0.125,both
4,luis.faria@novasbe.pt,1,2,0,0.150,0.000,3.000,105.000,1104 || Seminário de Economia Europeia || S1 |...,1,NaN,0.125,both


In [2096]:
# Check there are no BS courses without weight
final_market[(final_market["_merge"] != "both") & (final_market.masters_course==0)]

,TA,preference,preference_type,masters_course,new_contract,master_student,number_classes,number_students,course,semester,weight,weight_bs,_merge


In [2097]:
final_market.rename(columns={"new_contract": "capacity"}, inplace=True) # "weight": "bs_weight", 
final_market.drop(columns="_merge", inplace=True)

# ADDED! multiply by number of classes
final_market["weight"] = final_market["weight"].fillna(final_market["weight_bs"] * final_market["number_classes"])
final_market.drop(columns=["weight_bs"], inplace=True)

# final_market.loc[final_market['masters_course'] == 1, 'bs_capacity'] = np.nan
# final_market.loc[final_market['masters_course'] == 1, 'bs_weight'] = np.nan

In [2098]:
tas = final_market.TA.unique()

# Find elements missing in array1 compared to array2
missing_elements = np.setdiff1d(completed_preferences, tas)

# OUTPUT #N: TAs affected by unmatched courses
print(missing_elements)

['beatriz.f.ferreira@novasbe.pt' 'mariana.pires@novasbe.pt']


### Examples

In [2099]:
# final_market[final_market.TA=="bernardo.costa@novasbe.pt"].sort_values(by=['preference_type', 'preference'],ascending=[False,True])
final_market[final_market.TA=="fabian.wassmann@novasbe.pt"].sort_values(by=['preference_type', 'preference'],ascending=[False,True])

,TA,preference,preference_type,masters_course,capacity,master_student,number_classes,number_students,course,semester,weight
297,fabian.wassmann@novasbe.pt,1,2,1,0.125,0.000,1.000,80.000,2233 || Macroeconomics of Financial Markets ||...,0,0.174
323,fabian.wassmann@novasbe.pt,2,2,1,0.125,0.000,3.000,300.000,2269 || Empirical Methods for Finance || T1 || EN,0,0.651
415,fabian.wassmann@novasbe.pt,3,2,1,0.125,0.000,2.000,160.000,2434 || Statistics II || T1 || EN,0,0.347
427,fabian.wassmann@novasbe.pt,4,2,1,0.125,0.000,2.000,160.000,2435 || Statistics III || T1 || EN,0,0.347
256,fabian.wassmann@novasbe.pt,5,2,1,0.125,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130


In [2100]:
final_market[final_market.course=='2193 || Behavioral Economics and Finance || T1 || EN'].sort_values(by=['preference_type', 'preference'],ascending=[False,True]) # '1117 || Introdução à Microeconomia || S1 || EN'
# final_market[final_market.course=="2597 || Advanced Data Analysis || S1 || EN"].sort_values(by=['preference_type', 'preference'],ascending=[False,True])

,TA,preference,preference_type,masters_course,capacity,master_student,number_classes,number_students,course,semester,weight
262,priya.krishnan@novasbe.pt,1,2,1,0.500,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
261,iqra.hamid@novasbe.pt,2,2,1,0.500,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
259,luis.morais@novasbe.pt,3,2,1,0.300,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
258,galina.vysotskaya@novasbe.pt,4,2,1,0.250,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
263,joao.pato@novasbe.pt,4,2,1,0.250,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
256,fabian.wassmann@novasbe.pt,5,2,1,0.125,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
257,svitlana.sivitskaya@novasbe.pt,3,1,1,0.250,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130
260,ruben.bento@novasbe.pt,5,1,1,0.500,0.000,1.000,60.000,2193 || Behavioral Economics and Finance || T1...,0,0.130


In [2101]:
# final_market[final_market.course=='1117 || Introdução à Microeconomia || S1 || EN'].sort_values(by=['preference', 'preference_type'],ascending=[True,True])
# final_market[final_market.course=='1118 || Introdução à Macroeconomia || S1 || PT'].sort_values(by=['preference_type', 'preference'],ascending=[True,True])
final_market[final_market.course== '1318 || Programação || S1 || EN'].sort_values(by=['preference_type', 'preference'],ascending=[True,True])

,TA,preference,preference_type,masters_course,capacity,master_student,number_classes,number_students,course,semester,weight
190,paulo.rocha@novasbe.pt,2,0,0,0.500,0.000,2.000,90.000,1318 || Programação || S1 || EN,1,0.667
191,jose.ferrao@novasbe.pt,2,0,0,0.500,0.000,2.000,90.000,1318 || Programação || S1 || EN,1,0.667
189,marcio.silva@novasbe.pt,3,0,0,0.050,0.000,2.000,90.000,1318 || Programação || S1 || EN,1,0.667
192,paulo.marques@novasbe.pt,1,1,0,0.330,0.000,2.000,90.000,1318 || Programação || S1 || EN,1,0.667
193,bernardo.costa@novasbe.pt,1,1,0,0.333,0.000,2.000,90.000,1318 || Programação || S1 || EN,1,0.667


# Matching algorithm

In [2102]:
translation_mapping

{"Masters' Courses": 2, "Bachelors' Courses": 0, 'Indifferent': 1, NaT: 1}

In [2103]:
ta_dict = final_market[['TA','capacity']].drop_duplicates()
ta_dict = dict(zip(ta_dict['TA'], ta_dict['capacity']))

In [2104]:
# ADDED 'preference_type' == 2
ms_courses = final_market[(final_market['masters_course'] == 1) & (final_market['master_student'] == 0) & ((final_market['preference_type'] == 2) | (final_market['preference_type'] == 1)) & (final_market['preference'] == 1)]

ms_courses_dict = ms_courses[['course','weight']].drop_duplicates()
ms_courses_dict = dict(zip(ms_courses_dict['course'], ms_courses_dict['weight']))

# ADDED 'preference_type' == 2
bs_courses = final_market[(final_market['masters_course'] == 0) & ((final_market['preference_type'] == 0) | (final_market['preference_type'] == 1)) & (final_market['preference'] == 1)]

bs_courses_dict = bs_courses[['course','weight']].drop_duplicates()
bs_courses_dict = dict(zip(bs_courses['course'], bs_courses['weight']))

In [2105]:
ms_final_preferences = ms_courses[["TA", "preference_type", "preference", "course", "semester"]]
ms_final_preferences = ms_final_preferences.sort_values(by=["course", "preference_type", "preference"], ascending=[True, False, True])

bs_final_preferences = bs_courses[["TA", "preference_type", "preference", "course", "semester"]]
bs_final_preferences = bs_final_preferences.sort_values(by=["course", "preference_type", "preference"], ascending=[True, True, True])

In [2106]:
print(len(ms_final_preferences.course.unique()))
print(len(bs_final_preferences.course.unique()))

58
31


In [2107]:
# # Count the number of observations for each course and preference
# course_preference_counts = ms_final_preferences.groupby(['course', 'preference']).size().reset_index(name='count')
# # Filter out the courses with more than one observation for preference == 1
# filtered_courses = course_preference_counts.loc[(course_preference_counts['preference'] == 1) & (course_preference_counts['count'] == 1), 'course']
# # Filter the original DataFrame based on the filtered courses
# ms_final_preferences_filtered = ms_final_preferences[ms_final_preferences['course'].isin(filtered_courses)]

# # Print the filtered DataFrame
# print(len(ms_final_preferences_filtered.course.unique()))

In [2108]:
# # Count the number of observations for each course and preference
# course_preference_counts = bs_final_preferences.groupby(['course', 'preference']).size().reset_index(name='count')
# # Filter out the courses with more than one observation for preference == 1
# filtered_courses = course_preference_counts.loc[(course_preference_counts['preference'] == 1) & (course_preference_counts['count'] == 1), 'course']
# # Filter the original DataFrame based on the filtered courses
# bs_final_preferences_filtered = bs_final_preferences[bs_final_preferences['course'].isin(filtered_courses)]

# # Print the filtered DataFrame
# print(len(bs_final_preferences_filtered.course.unique()))

In [2109]:
ta_allocations = []

In [2110]:
for _, row in bs_final_preferences.iterrows():
    ta = row['TA']
    course = row['course']
    ta_capacity = ta_dict[ta]
    course_weight = bs_courses_dict[course]
    # Check if course can be allocated
    if course_weight > 0:
        # Check if TA still has capacity
        if  ta_capacity > 0:
            allocated_weight = min(course_weight, ta_capacity)
            # Allocate course to TA
            ta_allocations.append((ta, course, allocated_weight))
            ta_dict[ta] -= allocated_weight
            bs_courses_dict[course] -= allocated_weight
        else:
            try:
                bs_final_preferences = bs_final_preferences[bs_final_preferences['ta'] != ta]
            except:
                continue
    else:
        bs_final_preferences = bs_final_preferences[bs_final_preferences['course'] != course]

In [2111]:
for _, row in ms_final_preferences.iterrows():
    ta = row['TA']
    course = row['course']
    ta_capacity = ta_dict[ta]
    course_weight = ms_courses_dict[course]
    # Check if course can be allocated
    if course_weight > 0:
        # Check if TA still has capacity
        if  ta_capacity > 0:
            allocated_weight = min(course_weight, ta_capacity)
            # Allocate course to TA
            ta_allocations.append((ta, course, allocated_weight))
            ta_dict[ta] -= allocated_weight
            ms_courses_dict[course] -= allocated_weight
        else:
            try:
                ms_final_preferences = ms_final_preferences[ms_final_preferences['ta'] != ta]
            except:
                continue
    else:
        ms_final_preferences = ms_final_preferences[ms_final_preferences['course'] != course]

### Results

In [2112]:
len(ta_allocations)

133

In [2113]:
ta_allocations

[('vitor.cavalcante@novasbe.pt',
  '1108 || Economia do Desenvolvimento || S1 || EN',
  0.25),
 ('joao.ferreira@novasbe.pt', '1112 || Economia Industrial || S1 || EN', 0.25),
 ('ruben.bento@novasbe.pt', '1112 || Economia Industrial || S1 || EN', 0.125),
 ('diogo.lima@novasbe.pt', '1114 || Economia Global II || S1 || EN', 0.125),
 ('henrique.d.santos@novasbe.pt',
  '1115 || Economia Pública || S1 || EN',
  0.125),
 ('catarina.angelo@novasbe.pt',
  '1117 || Introdução à Microeconomia || S1 || EN',
  0.35),
 ('carlos.figueira@novasbe.pt',
  '1117 || Introdução à Microeconomia || S1 || EN',
  0.25),
 ('pedro.m.santos@novasbe.pt',
  '1117 || Introdução à Microeconomia || S1 || EN',
  0.375),
 ('mariana.m.videira@novasbe.pt',
  '1117 || Introdução à Microeconomia || S1 || EN',
  0.25),
 ('diogo.guerreiro@novasbe.pt',
  '1118 || Introdução à Macroeconomia || S1 || EN',
  0.25),
 ('joao.gama@novasbe.pt',
  '1118 || Introdução à Macroeconomia || S1 || PT',
  0.125),
 ('francisco.f.rodrigues@nov

In [2114]:
ms_courses_dict

{'2174 || Macroeconomic Analysis || S1 || EN': 0.0,
 '2175 || Econometrics || S1 || EN': 0.0,
 '2176 || Development Economics || S1 || EN': 0.0,
 '2188 || Microeconomic Analysis  || S1 || EN': 0.0,
 '2193 || Behavioral Economics and Finance || T1 || EN': 0.0,
 '2204 || Mergers, Acquisitions, Restructuring || T2 || EN': 0.0,
 '2215 || Auditing || T2 || EN': 0.0,
 '2217 || Corporate Governance || T1 || EN': 0.0,
 '2222 || Financial Statement Analysis || T1 || EN': 0.39583333333333337,
 '2229 || Investments || S1 || EN': 0.3557291666666671,
 '2233 || Macroeconomics of Financial Markets || T1 || EN': 0.0,
 '2238 || Financial Reporting || T1 || EN': 0.0,
 '2253 || Corporate Finance || S1 || EN': 0.7057291666666672,
 '2269 || Empirical Methods for Finance || T1 || EN': 0.42604166666666665,
 '2277 || Impact Investments || T1 || EN': 0.0,
 '2327 || Brand Management || S1 || EN': 0.0,
 '2332 || Entrepreneurship || T1 || EN': 0.0,
 '2336 || Human Resources Management || T1 || EN': 0.0,
 '2336 ||

In [2115]:
bs_courses_dict

{'1108 || Economia do Desenvolvimento || S1 || EN': 0.0,
 '1112 || Economia Industrial || S1 || EN': 0.0,
 '1114 || Economia Global II || S1 || EN': 0.125,
 '1115 || Economia Pública || S1 || EN': 0.0,
 '1117 || Introdução à Microeconomia || S1 || EN': 0.1499999999999999,
 '1118 || Introdução à Macroeconomia || S1 || PT': 0.0,
 '1118 || Introdução à Macroeconomia || S1 || EN': 1.0,
 '1119 || Microeconomia || S1 || EN': 0.2,
 '1120 || Macroeconomia || S1 || EN': 0.0,
 '1206 || Comportamento Organizacional || S1 || EN': 0.25,
 '1207 || Estratégia || S1 || EN': 0.0,
 '1208 || Sistemas de Informação || S1 || EN': 0.075,
 '1210 || Empreendedorismo || S1 || EN': 0.0,
 '1211 || Gestão Internacional || S1 || EN': 0.04999999999999999,
 '1212 || Contexto Internacional da Gestão || S1 || EN': 0.0,
 '1217 || Contabilidade Financeira || S1 || EN': 1.0,
 '1218 || Contabilidade de Gestão || S1 || EN': 0.625,
 '1219 || Finanças || S1 || EN': 0.0,
 '1220 || Marketing || S1 || EN': 0.125,
 '1221 || Gest

In [2116]:
# ms_courses_dict_filtered = {k: v for k, v in ms_courses_dict.items() if v == 0}
# len(ms_courses_dict_filtered)

In [2117]:
# ms_courses_dict_filtered = {k: v for k, v in ms_ta_dict.items() if v < 5}
# len(ms_courses_dict_filtered)
# ms_courses_dict_filtered

### Version combined

In [2118]:
# courses_dict = final_market[['course','weight']].drop_duplicates()
# courses_dict = dict(zip(courses_dict['course'], courses_dict['weight']))

In [2119]:
# course_preference_counts = final_preferences.groupby(['course', 'preference']).size().reset_index(name='count')

# # Filter out the courses with more than one observation for preference == 1
# filtered_courses = course_preference_counts.loc[(course_preference_counts['preference'] == 1) & (course_preference_counts['count'] == 1), 'course']
# final_preferences_filtered = final_preferences[final_preferences['course'].isin(filtered_courses)]

# # Print the filtered DataFrame
# print(len(final_preferences_filtered.course.unique()))

In [2120]:
# final_preferences = final_market[["TA", "preference_type", "preference", "course", "semester", "master_student", "masters_course"]]
# final_preferences = final_preferences.sort_values(by=["course", "preference_type", "preference"], ascending=[True, False, True])
# final_preferences.head()

In [2121]:
# len(final_preferences.course.unique())

In [2122]:
# ta_allocations = []

# for _, row in final_preferences_filtered.iterrows():
#     ta = row['TA']
#     pref_type = row['preference_type']
#     course = row['course']
#     student = row['master_student']
#     ms_course = row['masters_course']

#     ta_capacity = ta_dict[ta]
#     course_weight = courses_dict[course]

#     # MS courses
#     if ms_course == 1 and student == 0:
#         # Give priority to TAs who want MS courses
#         if pref_type == 1 or pref_type == 2:
#             # Check if course can be allocated
#             if course_weight > 0:
#                 # Check if TA still has capacity
#                 if  ta_capacity > 0:
#                     allocated_weight = min(course_weight, ta_capacity)
#                     # Allocate course to TA
#                     ta_allocations.append((ta, course, allocated_weight))
#                     ta_dict[ta] -= allocated_weight
#                     courses_dict[course] -= allocated_weight
#                 else:
#                     try:
#                         final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['ta'] != ta]
#                     except:
#                         continue
#             else:
#                 final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['course'] != course]
#         else:
#             continue
    
#     elif ms_course == 0:
#         # Give priority to TAs who want BS courses
#         if pref_type == 0:
#             # Check if course can be allocated
#             if course_weight > 0:
#                 # Check if TA still has capacity
#                 if  ta_capacity > 0:
#                     allocated_weight = min(course_weight, ta_capacity)
#                     # Allocate course to TA
#                     ta_allocations.append((ta, course, allocated_weight))
#                     ta_dict[ta] -= allocated_weight
#                     courses_dict[course] -= allocated_weight
#                 else:
#                     try:
#                         final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['ta'] != ta]
#                     except:
#                         continue
#             else:
#                 final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['course'] != course]
#         else:
#             continue
#     else:
#         if course_weight > 0:
#             # Check if TA still has capacity
#             if  ta_capacity > 0:
#                 allocated_weight = min(course_weight, ta_capacity)
#                 # Allocate course to TA
#                 ta_allocations.append((ta, course, allocated_weight))
#                 ta_dict[ta] -= allocated_weight
#                 courses_dict[course] -= allocated_weight
#             else:
#                 try:
#                     final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['ta'] != ta]
#                 except:
#                     continue
#         else:
#             final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['course'] != course]            